# BackT Tutorial: Long-Short Moving Average Strategy

This notebook demonstrates how to implement and test a long-short moving average crossover strategy using the BackT framework with mock data. Unlike the long-only version, this strategy goes short on death crosses, potentially profiting in bear markets.

## Overview
1. **Setup & Imports** - Import required libraries
2. **Universe Definition** - Define the assets to trade
3. **Date Configuration** - Set up the backtesting period
4. **Long-Short Strategy** - Create moving average strategy with short positions
5. **Mock Data Generation** - Generate synthetic market data
6. **Engine Configuration** - Configure the BackT engine
7. **Run Backtest** - Execute the strategy
8. **Results Analysis** - Analyze performance metrics
9. **Visualization** - Plot equity curves and results
10. **Long vs Long-Short Comparison** - Compare against long-only approach
11. **Market Scenario Testing** - Test across different market conditions

## 1. Setup & Imports

First, let's import all the necessary components from BackT and other required libraries.

In [55]:
# Add parent directory to path (if running from notebooks folder)
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

# Import BackT components
from backt import Backtester, BacktestConfig, MockDataLoader
from backt.signal import TechnicalIndicators, StrategyHelpers
from backt.utils.config import ExecutionConfig

# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import Dict, Any

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All imports successful!")
print("📊 Ready for long-short backtesting with BackT")
print("⚡ This strategy can go LONG and SHORT!")

✅ All imports successful!
📊 Ready for long-short backtesting with BackT
⚡ This strategy can go LONG and SHORT!


## 2. Universe Definition

Define the universe of assets we want to trade. For long-short strategies, we want assets with good liquidity and borrowability.

In [57]:
# Define our trading universe
universe = {
    'SPY': 'US Large Cap Stocks (S&P 500)',
    'QQQ': 'US Tech Stocks (NASDAQ 100)',
    'TLT': 'Long-term US Treasuries (20+ Year)',
    'GLD': 'Gold ETF',
    # 'IWM': 'US Small Cap Stocks (Russell 2000)',
    # 'EFA': 'International Developed Markets',
    # 'EEM': 'iShares MSCI Emerging Markets ETF',
    # 'FXI': 'iShares China Large-Cap ETF',
    'IEF': 'iShares 7–10 Year Treasury ETF',
    'SHY': 'iShares 1–3 Year Treasury ETF',
    # 'HYG': 'iShares iBoxx High Yield Corporate Bond ETF',
    # 'USO': 'United States Oil Fund',
    'DBC': 'Invesco DB Commodity Index Tracking Fund',
}

# Extract just the symbols for trading
symbols = list(universe.keys())

# Display our universe
print("🌍 Long-Short Trading Universe:")
print("=" * 50)
for symbol, description in universe.items():
    print(f"{symbol:4} - {description}")
    
print(f"\n📊 Total assets: {len(symbols)}")
print(f"🎯 Symbols for long-short trading: {symbols}")
print(f"📈 Can go LONG on golden crosses")
print(f"📉 Can go SHORT on death crosses")

🌍 Long-Short Trading Universe:
SPY  - US Large Cap Stocks (S&P 500)
QQQ  - US Tech Stocks (NASDAQ 100)
TLT  - Long-term US Treasuries (20+ Year)
GLD  - Gold ETF
IEF  - iShares 7–10 Year Treasury ETF
SHY  - iShares 1–3 Year Treasury ETF
DBC  - Invesco DB Commodity Index Tracking Fund

📊 Total assets: 7
🎯 Symbols for long-short trading: ['SPY', 'QQQ', 'TLT', 'GLD', 'IEF', 'SHY', 'DBC']
📈 Can go LONG on golden crosses
📉 Can go SHORT on death crosses


## 3. Date Configuration

Set up the backtesting period and other time-related parameters.

In [59]:
# Define backtesting period
start_date = '2018-01-01'
end_date = '2025-9-27'
initial_capital = 100000.0

# Calculate period length
start_dt = pd.to_datetime(start_date)
end_dt = pd.to_datetime(end_date)
period_days = (end_dt - start_dt).days
period_years = period_days / 365.25

print("📅 Long-Short Backtesting Configuration:")
print("=" * 45)
print(f"Start Date:      {start_date}")
print(f"End Date:        {end_date}")
print(f"Period:          {period_days} days ({period_years:.1f} years)")
print(f"Initial Capital: ${initial_capital:,.0f}")
print(f"Data Frequency:  Daily")
print(f"Strategy Type:   Long-Short (can short on death cross)")

# Store configuration for later use
backtest_params = {
    'start_date': start_date,
    'end_date': end_date,
    'initial_capital': initial_capital,
    'period_years': period_years
}

print("\n✅ Date configuration complete!")

📅 Long-Short Backtesting Configuration:
Start Date:      2018-01-01
End Date:        2025-9-27
Period:          2826 days (7.7 years)
Initial Capital: $100,000
Data Frequency:  Daily
Strategy Type:   Long-Short (can short on death cross)

✅ Date configuration complete!


## 4. Long-Short Moving Average Strategy Definition

Create a long-short moving average crossover strategy that can take both long and short positions.

In [61]:
from backt.signal.kalman import KalmanFilter1D

def long_short_moving_average_strategy(
    market_data: Dict[str, pd.DataFrame],
    current_time: pd.Timestamp,
    positions: Dict[str, Any],
    context: Dict[str, Any],
    params: Dict[str, Any]
) -> Dict[str, Dict]:
    """
    Kalman-Enhanced Long-Short Moving Average Crossover Strategy
    
    Logic:
    - Use Kalman filters instead of traditional moving averages for denoising
    - Two filters with different Q parameters act as fast/slow signals
    - LONG when fast Kalman > slow Kalman (trend confirmation)
    - SHORT when fast Kalman < slow Kalman (downtrend confirmation)
    - Equal weight allocation across selected positions (long and short)
    
    Key Advantages of Kalman Filters:
    - Superior noise reduction without lag
    - Adaptive to changing market conditions
    - No lookback window needed (real-time adaptation)
    - Smoother signals with fewer whipsaws
    
    Parameters:
    - Q_fast: Process noise for fast filter (default: 0.01) - more responsive
    - Q_slow: Process noise for slow filter (default: 0.001) - smoother
    - R: Measurement noise (default: 1.0) - standard
    - min_periods: Minimum data points required (default: 60)
    - max_position_size: Maximum weight per position (default: 0.25)
    """
    
    # Strategy parameters
    Q_fast = params.get('Q_fast', 0.01)
    Q_slow = params.get('Q_slow', 0.001)
    R = params.get('R', 1.0)
    min_periods = params.get('min_periods', 60)
    max_position_size = params.get('max_position_size', 0.25)
    
    # Initialize Kalman filters in context if not already present
    if 'kalman_filters' not in context:
        context['kalman_filters'] = {}
    
    orders = {}
    signals = {}
    long_positions = []
    short_positions = []
    
    # Analyze each asset
    for symbol, data in market_data.items():
        if len(data) < min_periods:
            continue
            
        try:
            # Initialize filters for this symbol if needed
            if symbol not in context['kalman_filters']:
                context['kalman_filters'][symbol] = {
                    'fast': KalmanFilter1D(Q=Q_fast, R=R),
                    'slow': KalmanFilter1D(Q=Q_slow, R=R),
                    'prev_fast': None,
                    'prev_slow': None
                }
            
            filters = context['kalman_filters'][symbol]
            
            # Apply Kalman filters to the entire price series
            # We need to update the filters with all historical data
            prices = data['close']
            
            # Reset filters and recompute (stateful approach)
            # In production, you'd only update with new data, but for backtesting
            # we recompute to ensure consistency
            kf_fast = KalmanFilter1D(Q=Q_fast, R=R)
            kf_slow = KalmanFilter1D(Q=Q_slow, R=R)
            
            fast_values = []
            slow_values = []
            
            for price in prices:
                fast_values.append(kf_fast.update(price))
                slow_values.append(kf_slow.update(price))
            
            # Get current and previous Kalman estimates
            if len(fast_values) < 2 or len(slow_values) < 2:
                continue
            
            current_fast = fast_values[-1]
            current_slow = slow_values[-1]
            prev_fast = fast_values[-2]
            prev_slow = slow_values[-2]
            
            # Store current values for next iteration
            filters['prev_fast'] = current_fast
            filters['prev_slow'] = current_slow
            
            # Detect crossovers using Kalman-filtered values
            golden_cross = (prev_fast <= prev_slow) and (current_fast > current_slow)
            death_cross = (prev_fast >= prev_slow) and (current_fast < current_slow)
            
            # Generate signals - SHORT on death cross, LONG on golden cross
            if golden_cross:
                signals[symbol] = 'BUY'  # Golden cross = Go long
                long_positions.append(symbol)
            elif death_cross:
                signals[symbol] = 'SELL_SHORT'  # Death cross = Go short
                short_positions.append(symbol)
            elif current_fast > current_slow:
                signals[symbol] = 'HOLD_LONG'  # Fast above slow = Stay long
                long_positions.append(symbol)
            else:
                signals[symbol] = 'HOLD_SHORT'  # Fast below slow = Stay short
                short_positions.append(symbol)
                
        except Exception as e:
            print(f"Warning: Error processing {symbol}: {e}")
            continue
    
    # Calculate position sizing
    total_positions = len(long_positions) + len(short_positions)
    
    if total_positions > 0:
        # Equal weight allocation, but respect max position size
        weight_per_position = min(1.0 / total_positions, max_position_size)
        
        # Create LONG orders
        for symbol in long_positions:
            orders[symbol] = {
                'action': 'target_weight',
                'weight': weight_per_position  # Positive weight = LONG
            }
        
        # Create SHORT orders  
        for symbol in short_positions:
            orders[symbol] = {
                'action': 'target_weight',
                'weight': -weight_per_position  # Negative weight = SHORT
            }
    
    # Close positions for assets with no signal
    for symbol in market_data.keys():
        if symbol not in (long_positions + short_positions):
            if symbol in positions and hasattr(positions[symbol], 'quantity'):
                if positions[symbol].quantity != 0:
                    orders[symbol] = {'action': 'close'}
    
    # Store strategy state for analysis
    context['signals'] = signals
    context['long_positions'] = long_positions
    context['short_positions'] = short_positions
    context['num_long'] = len(long_positions)
    context['num_short'] = len(short_positions)
    context['total_positions'] = total_positions
    context['position_weight'] = weight_per_position if total_positions > 0 else 0
    
    return orders

# Define strategy parameters - KALMAN FILTER PARAMETERS
strategy_params = {
    'Q_fast': 0.01,             # Fast filter - more responsive (like 8-day MA)
    'Q_slow': 0.001,            # Slow filter - smoother (like 20-day MA)
    'R': 1.0,                   # Measurement noise
    'min_periods': 60,          # Minimum data points required
    'max_position_size': 0.25   # Maximum 25% in any single position
}

print("📈 Kalman-Enhanced Long-Short Strategy Configuration:")
print("=" * 50)
print(f"Strategy Type:      Kalman Filter MA Crossover (Long-Short)")
print(f"Fast Filter (Q):    {strategy_params['Q_fast']} (responsive)")
print(f"Slow Filter (Q):    {strategy_params['Q_slow']} (smooth)")
print(f"Measurement (R):    {strategy_params['R']}")
print(f"Min Periods:        {strategy_params['min_periods']} days")
print(f"Max Position Size:  {strategy_params['max_position_size']:.0%}")
print(f"Position Sizing:    Equal weight across long and short")

print("\n🎯 KEY STRATEGY FEATURES:")
print("-" * 40)
print("📊 Uses Kalman filters instead of traditional MAs")
print("📈 GOLDEN CROSS (Fast Kalman > Slow Kalman):  GO LONG")
print("📉 DEATH CROSS (Fast Kalman < Slow Kalman):   GO SHORT")
print("⚡ Adaptive noise reduction without lag")
print("🔬 Fewer false signals vs traditional MA crossover")
print("⚠️  Higher risk due to leverage and short positions")

print("\n✅ Kalman-enhanced long-short strategy definition complete!")

📈 Kalman-Enhanced Long-Short Strategy Configuration:
Strategy Type:      Kalman Filter MA Crossover (Long-Short)
Fast Filter (Q):    0.01 (responsive)
Slow Filter (Q):    0.001 (smooth)
Measurement (R):    1.0
Min Periods:        60 days
Max Position Size:  25%
Position Sizing:    Equal weight across long and short

🎯 KEY STRATEGY FEATURES:
----------------------------------------
📊 Uses Kalman filters instead of traditional MAs
📈 GOLDEN CROSS (Fast Kalman > Slow Kalman):  GO LONG
📉 DEATH CROSS (Fast Kalman < Slow Kalman):   GO SHORT
⚡ Adaptive noise reduction without lag
🔬 Fewer false signals vs traditional MA crossover
⚠️  Higher risk due to leverage and short positions

✅ Kalman-enhanced long-short strategy definition complete!


## 5. Mock Data Generation

Generate synthetic market data for testing our long-short strategy.

In [63]:
# Test data generation first
print("🔄 Generating mock data for long-short testing...")

# Create mock data loader
mock_loader = MockDataLoader(scenario='normal', seed=42)

# Generate data for our universe
mock_data = mock_loader.load(
    symbols=symbols,
    start_date=start_date,
    end_date=end_date
)

print("📊 Mock Data Summary:")
print("=" * 40)

for symbol, data in mock_data.items():
    start_price = data['close'].iloc[0]
    end_price = data['close'].iloc[-1]
    total_return = (end_price / start_price) - 1
    volatility = data['close'].pct_change().std() * np.sqrt(252)  # Annualized volatility
    
    print(f"{symbol}: {len(data)} days, {total_return:+6.1%} return, {volatility:5.1%} vol")

print(f"\n✅ Mock data generated for {len(mock_data)} assets")
print(f"📈 Data range: {data.index[0].date()} to {data.index[-1].date()}")
print(f"🎯 Perfect for testing long-short strategies!")
print(f"📉 Note: Short positions can profit from negative returns (GLD, IWM)")

🔄 Generating mock data for long-short testing...
📊 Mock Data Summary:
SPY: 2827 days, +146.9% return, 15.8% vol
QQQ: 2827 days, +257.6% return, 20.0% vol
TLT: 2827 days, +83.5% return,  8.2% vol
GLD: 2827 days, +133.2% return, 18.2% vol
IEF: 2827 days, -21.1% return, 16.2% vol
SHY: 2827 days, +288.9% return, 16.1% vol
DBC: 2827 days, +267.2% return, 15.5% vol

✅ Mock data generated for 7 assets
📈 Data range: 2018-01-01 to 2025-09-27
🎯 Perfect for testing long-short strategies!
📉 Note: Short positions can profit from negative returns (GLD, IWM)


## 6. Engine Configuration

Configure the BackT backtesting engine for long-short trading with appropriate costs.

In [65]:
# Configure execution settings (higher costs for short selling)
execution_config = ExecutionConfig(
    spread=0.00,                # 0.2% bid-ask spread (higher for short selling)
    slippage_pct=0.00,         # 0.1% slippage
    commission_per_share=0.0,    # Commission-free trading
    commission_per_trade=0.0     # No flat fees
)

# Create main backtest configuration
config = BacktestConfig(
    start_date=start_date,
    end_date=end_date,
    initial_capital=initial_capital,
    data_frequency='1D',         # Daily data
    execution=execution_config,
    
    # Mock data settings
    use_mock_data=False,          # Enable mock data mode
    mock_scenario='normal',      # Normal market conditions
    mock_seed=42,                # Reproducible results
    
    # Other settings
    verbose=True,                # Detailed logging
    save_trades=True,            # Save trade history
    save_positions=True,         # Save position history
    save_equity_curve=True,       # Save equity curve
    max_leverage=None,
    max_position_size=None,
)

print("⚙️ Long-Short Engine Configuration:")
print("=" * 40)
print(f"Mock Data Mode:    {config.use_mock_data}")
print(f"Mock Scenario:     {config.mock_scenario}")
print(f"Random Seed:       {config.mock_seed}")
print(f"Spread:            {execution_config.spread:.2%} (higher for short selling)")
print(f"Slippage:          {execution_config.slippage_pct:.2%}")
print(f"Commission:        ${execution_config.commission_per_trade}")
print(f"Strategy Type:     Long-Short (can short sell)")

print(f"\n💡 LONG-SHORT CONSIDERATIONS:")
print(f"-" * 35)
print(f"• Higher transaction costs due to short selling complexity")
print(f"• Short positions have unlimited loss potential")
print(f"• Can profit in both rising and falling markets")
print(f"• Requires margin account and borrowing capability")

print(f"\n✅ BackT engine configured for long-short trading!")
print(f"🎯 Ready to test long-short moving average strategy")

⚙️ Long-Short Engine Configuration:
Mock Data Mode:    False
Mock Scenario:     normal
Random Seed:       42
Spread:            0.00% (higher for short selling)
Slippage:          0.00%
Commission:        $0.0
Strategy Type:     Long-Short (can short sell)

💡 LONG-SHORT CONSIDERATIONS:
-----------------------------------
• Higher transaction costs due to short selling complexity
• Short positions have unlimited loss potential
• Can profit in both rising and falling markets
• Requires margin account and borrowing capability

✅ BackT engine configured for long-short trading!
🎯 Ready to test long-short moving average strategy


## 7. Run Long-Short Backtest

Execute the long-short moving average crossover strategy on our mock data.

In [67]:
print("🚀 Starting Long-Short Backtest...")
print("=" * 45)
print(f"Strategy: Long-Short Moving Average Crossover")
print(f"Positions: Can go LONG or SHORT")
print(f"Assets: {len(symbols)} ETFs")

# Initialize backtester
backtester = Backtester(config)

# Run the backtest
start_time = datetime.now()

result = backtester.run(
    strategy=long_short_moving_average_strategy,
    universe=symbols,
    strategy_params=strategy_params
)

end_time = datetime.now()
runtime = (end_time - start_time).total_seconds()

print(f"\n🎉 Long-Short backtest completed!")
print(f"⏱️  Execution time: {runtime:.2f} seconds")
print(f"📊 Results ready for analysis")
print(f"⚡ Strategy used both LONG and SHORT positions")

🚀 Starting Long-Short Backtest...
Strategy: Long-Short Moving Average Crossover
Positions: Can go LONG or SHORT
Assets: 7 ETFs
2025-10-12 22:46:39,347 - backt.Backtester - INFO - Using Yahoo Finance data loader
2025-10-12 22:46:39,352 - backt.Backtester - INFO - Starting backtest
2025-10-12 22:46:39,352 - backt.Backtester - INFO - Loading data for 7 symbols
2025-10-12 22:46:39,352 - backt.YahooDataLoader - INFO - Loading Yahoo Finance data for ['SPY', 'QQQ', 'TLT', 'GLD', 'IEF', 'SHY', 'DBC']


[*********************100%***********************]  7 of 7 completed

2025-10-12 22:46:40,480 - backt.Backtester - INFO - Backtesting 1945 periods from 2018-01-02 00:00:00+00:00 to 2025-09-26 00:00:00+00:00
2025-10-12 22:46:40,487 - backt.Backtester - INFO - Progress: 0.1% - 2018-01-02 - Equity: $100,000.00


2025-10-12 22:46:40,947 - backt.Backtester - INFO - Progress: 5.2% - 2018-05-25 - Equity: $98,388.52
2025-10-12 22:46:41,744 - backt.Backtester - INFO - Progress: 10.3% - 2018-10-17 - Equity: $99,571.73
2025-10-12 22:46:42,642 - backt.Backtester - INFO - Progress: 15.5% - 2019-03-14 - Equity: $101,501.33
2025-10-12 22:46:43,524 - backt.Backtester - INFO - Progress: 20.6% - 2019-08-06 - Equity: $103,524.39
2025-10-12 22:46:44,428 - backt.Backtester - INFO - Progress: 25.8% - 2019-12-27 - Equity: $104,951.44
2025-10-12 22:46:45,334 - backt.Backtester - INFO - Progress: 30.9% - 2020-05-21 - Equity: $111,794.81
2025-10-12 22:46:46,256 - backt.Backtester - INFO - Progress: 36.0% - 2020-10-13 - Equity: $120,461.40
2025-10-12 22:46:47,181 - backt.Backtester - INFO - Progress: 41.2% - 2021-03-09 - Equity: $131,111.39
2025-10-12 22:46:48,067 - backt.Backtester - INFO - Progress: 46.3% - 2021-07-30 - Equity: $138,132.77
2025-10-12 22:46:48,973 - backt.Backtester - INFO - Progress: 51.5% - 2021-1

## Summary

🎯 **What We Accomplished:**

1. ✅ **Implemented a Long-Short Moving Average Strategy** - Can go both long and short
2. ✅ **Added short selling capability** - Death crosses trigger short positions instead of cash
3. ✅ **Enhanced strategy framework** - Equal weight allocation across long and short positions
4. ✅ **Comprehensive testing setup** - Ready for comparison with long-only approaches

🚀 **Key Features:**

- **Market Neutrality**: Can potentially profit in both rising and falling markets
- **Enhanced Signals**: Death crosses become profit opportunities, not just exit signals
- **Risk Management**: Position sizing controls across both long and short positions
- **Realistic Execution**: Higher transaction costs for short selling complexity

This notebook provides the foundation for exploring long-short momentum strategies and comparing their performance characteristics against simpler long-only approaches!

## Using PerformanceReport - Standard Reporting

Now let's use the new **PerformanceReport** module for standardized, reusable reporting.

In [70]:
# Import the PerformanceReport module
from backt.reporting import PerformanceReport, ReportConfig

# Create and print standard report
report = PerformanceReport(result, initial_capital=backtest_params['initial_capital'])
report.print_report()

2025-10-12 22:46:59,953 - backt.PerformanceReport - INFO - Fetching benchmark data for SPY...
2025-10-12 22:47:00,026 - backt.PerformanceReport - INFO - Benchmark loaded: 176.60% total return
📊 PERFORMANCE ANALYSIS

📈 RETURN METRICS:
-------------------------
Total Return:               57.17%
Annualized Return:           1.27%
CAGR:                        1.22%

⚠️  RISK METRICS:
--------------------
Volatility:                  3.59%
Sharpe Ratio:                0.355
Sortino Ratio:               0.463
Maximum Drawdown:          -15.20%
Calmar Ratio:                0.080

📊 TRADING ACTIVITY:
------------------------
Total Trades:                13202
Win Rate:                    75.8%
Profit Factor:                3.47
Average Trade:               0.00%

💼 PORTFOLIO SUMMARY:
-------------------------
Initial Capital:          $   100,000
Final Value:              $   157,168
Profit/Loss:              $    57,168

🎯 PERFORMANCE ASSESSMENT:
------------------------------
Sharpe Rating:

In [71]:
# Generate all standard charts
report.show_charts()

In [72]:
# Get metrics as DataFrame for export or further analysis
metrics_df = report.get_metrics_dataframe(transpose=True)
print("Metrics DataFrame:")
metrics_df.head(10)  # Show first 10 metrics

Metrics DataFrame:


Strategy  SPY (Buy & Hold)
total_return           0.571683          1.765972
cagr                   0.012162          0.141054
annualized_volatility  0.035896          0.196607
volatility             0.035896          0.196607
annualized_return      0.012739          0.141054
sharpe_ratio           0.354898          0.717441
sortino_ratio          0.462887          0.875998
annual_return          0.012739          0.141054
best_day               0.054877          0.105019
worst_day             -0.033026         -0.109423